## 🎯Data Cleaning & Preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('nigeria-rent.csv')
df.head(0)

In [ ]:
df.rename(columns = {'More Info':'Description'}, inplace = True)
df.rename(columns = {'Title':'Real Estate Agent Tag'}, inplace = True)

In [ ]:
## Removing non-residential properties

# Search "Description" Column for Keywords
df['Description'].unique()

# Remove all observations that are in any way non-residential property by searching for red-flag keywords
df_a1 = df.loc[df['Description'].str.contains('COMMERCIAL', case=False)]
df_a2 = pd.concat([df,df_a1]).drop_duplicates(keep=False)

df_b1 = df_a2.loc[df['Description'].str.contains('OFFICE', case=False)]
df_b2 = pd.concat([df_a2,df_b1]).drop_duplicates(keep=False)

df_c1 = df_b2.loc[df['Description'].str.contains('WORKING', case=False)]
df_c2 = pd.concat([df_b2,df_c1]).drop_duplicates(keep=False)

df_d1 = df_c2.loc[df['Description'].str.contains('LAND', case=False)]
df_d2 = pd.concat([df_c2,df_d1]).drop_duplicates(keep=False)

df_e1 = df_d2.loc[df['Description'].str.contains('JOINT VENTURE', case=False)]
df_e2 = pd.concat([df_d2,df_e1]).drop_duplicates(keep=False)

df_f1 = df_e2.loc[df['Description'].str.contains('CONFERENCE', case=False)]
df_f2 = pd.concat([df_e2,df_f1]).drop_duplicates(keep=False)

# This isn't totally sufficient; let's also only keep observations with "Bedroom" in the description string
df_cd = df_f2.loc[df['Description'].str.contains('BEDROOM', case=False)]
df_cd['Description'].unique()


In [ ]:
# Extracting additoinal information from the 'Description' column

# Create binary true/false columns for 'Duplex', 'Terraced', 'Detached' and 'Penthouse'
df_duplex = df_cd.loc[df_cd['Description'].str.contains('DUPLEX', case=False)]
duplexindexes = df_duplex.index.to_list()
df_cd['Duplex'] = 0
df_cd.loc[duplexindexes, 'Duplex'] = 1

df_terraced = df_cd.loc[df_cd['Description'].str.contains('TERRACED', case=False)]
terracedindexes = df_terraced.index.to_list()
df_cd['Terraced'] = 0
df_cd.loc[terracedindexes, 'Terraced'] = 1

df_detached = df_cd.loc[df_cd['Description'].str.contains('DETACHED', case=False)]
detachedindexes = df_detached.index.to_list()
df_cd['Detached'] = 0
df_cd.loc[detachedindexes, 'Detached'] = 1

df_penthouse = df_cd.loc[df_cd['Description'].str.contains('PENTHOUSE', case=False)]
penthouseindexes = df_penthouse.index.to_list()
df_cd['Penthouse'] = 0
df_cd.loc[penthouseindexes, 'Penthouse'] = 1

df_apartment = df_cd.loc[df_cd['Description'].str.contains('APARTMENT', case=False)]
df_flat = df_cd.loc[df_cd['Description'].str.contains('FLAT', case=False)]
apartmentindexes = df_apartment.index.to_list()
flatindexes = df_flat.index.to_list()
df_cd['Apartment'] = 0
df_cd.loc[apartmentindexes, 'Apartment'] = 1
df_cd.loc[flatindexes, 'Apartment'] = 1

# Sidenote: all entries that are not apartments or flats must be houses, hence: 
df_cd['House'] = 1
df_cd.loc[apartmentindexes, 'House'] = 0

# Search the Dataframe for attachment keywords and drop all that have no valid attachment type (terraced, detached, semi-detached)
df_detached = df_cd.loc[df_cd['Description'].str.contains('DETACHED', case=False)]
detachedindexes2 = df_detached.index.to_list()
df_terraced = df_cd.loc[df_cd['Description'].str.contains('TERRACED', case=False)]
terracedindexes2 = df_terraced.index.to_list()
attachmenttype = detachedindexes2 + terracedindexes2
df_clean = df_cd.loc[attachmenttype]


In [ ]:
# Eliminating penthouse column

# The above cleaning cell removed all observations that were penthouses, so this feature is irrelevant now. 
df_clean['Penthouse'].unique()
df_clean.drop('Penthouse', axis=1)

In [ ]:
# Dropping description and real estate agent tag columns

# We've gotten everything we need from these columns and they won't be useful for our ML model. 
df_clean.drop(['Real Estate Agent Tag', 'Description'], axis=1, inplace=True)

In [ ]:
# Converting to integers
df_clean['Bedrooms'].replace(' beds', '', regex=True, inplace=True)
df_clean['Bedrooms'] = pd.to_numeric(df_clean['Bedrooms']).astype('Int64')

df_clean['Bathrooms'].replace(' baths', '', regex=True, inplace=True)
df_clean['Bathrooms'] = pd.to_numeric(df_clean['Bathrooms']).astype('Int64')

df_clean['Toilets'].replace(' Toilets', '', regex=True, inplace=True)
df_clean['Toilets'] = pd.to_numeric(df_clean['Toilets']).astype('Int64')

df_clean['Price'].replace(',', '', regex=True, inplace=True)
df_clean['Price'] = df_clean.Price.str.extract('(^\d*)') # Doesn't seem like it's needed, maybe add back later
df_clean['Price'] = pd.to_numeric(df_clean['Price'])

# Code creates some NaN values for the "Bathrooms" and "Toilets" columns. Let's clean. 
df_clean.dropna(inplace=True)

In [ ]:
# Cleaning outliers
# Let's clean outliers in terms of price to enhance the predictive capability of the model. 
# We attempt to use the standard 1.5*IQA method as high- and lowpass filters, but the low cutoff is then in negative price territory. 
# We make a judgement call and implement a cut of 20'000 Naira on the low end. 

Q1 = np.quantile(df_clean['Price'], 0.25)
Q3 = np.quantile(df_clean['Price'], 0.75)
Tolerance = 1.5*(Q3 - Q1)
LoCut = Q1 - Tolerance
HiCut = Q3 + Tolerance

df_clean1 = df_clean.loc[df_clean['Price'] > 20000]
df_clean2 = df_clean1.loc[df_clean['Price'] < HiCut]
df_clean2['Price'].unique()


In [ ]:
# Location cleaning
# Let's make the location column somewhat more useable by splitting it into state, area, and microlocation features.
# Further geographical work can be found in the dedicated (GeoData/GeoPlots) Jupyter notebook

# New column for states
df_clean2["State"] = df_clean2["Location"].str.split().str[-1]

# From exploratory data analysis, we see that Lagos is by far the most-represented area in the dataset. We focus on Lagos only. 
df_clean2 = df_clean2.loc[df_clean2["State"]=="Lagos"]

#New column for Areas
df_clean2["Area"] =  np.where(df_clean2.Location.str.contains("Victoria Island"), "Victoria Island",
                      np.where(df_clean2.Location.str.contains("Lagos Island"), "Ikoyi",
                      np.where(df_clean2.Location.str.contains("Ojota"), "Ogudu",
                       df_clean2["Location"].str.split().str[-2])))
#New column for microlocations
df_clean2["Microlocation"] = df_clean2["Location"].str.rsplit(' ', 2).str[0]

# We can drop the obsolete 'Location' Column. 
df_clean2.drop(['Location'], axis=1, inplace=True) 

# Let's create dummy variables for the 'Area' column
Listte = np.arange(1,37)

Area_number = {}
for i,x in zip(df_clean2.Area.unique(),Listte ):
    Area_number[i] = x
    
Area_dummy = []
for i in range(len(df_clean2['Area'])):
    Area_dummy.append(Area_number[df_clean2['Area'].values[i]])
    
df_clean2['Area Dummy'] = pd.DataFrame(Area_dummy)

In [ ]:
# Exporting

# Index Cleaning
df_clean2 = df_clean2.reset_index()
df_clean2.drop(['index'], axis=1, inplace=True) 

# Export
df_clean2.to_csv('NigeriaCleanedFinal.csv')